In [2]:
#Import splinter and beautiful soup

from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

In [5]:
executable_path = {'executable_path': ChromeDriverManager().install()}

#Double astericks(**kwargs) indicate passing x amounts of functions through the variable after the astericks
#*args is used to pass a non-keyworded variable-length argument list to your function. 
#**kwargs lets you pass a keyworded variable-length of arguments to your function.

browser = Browser('chrome', **executable_path, headless = False)



====== WebDriver manager ======
Could not get version for google-chrome with the command:  powershell "$ErrorActionPreference='silentlycontinue' ; (Get-Item -Path "$env:PROGRAMFILES\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion ; if (-not $? -or $? -match $error) { (Get-Item -Path "$env:PROGRAMFILES(x86)\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { (Get-Item -Path "$env:LOCALAPPDATA\Google\Chrome\Application\chrome.exe").VersionInfo.FileVersion } if (-not $? -or $? -match $error) { reg query "HKCU\SOFTWARE\Google\Chrome\BLBeacon" /v version } if (-not $? -or $? -match $error) { reg query "HKLM\SOFTWARE\Wow6432Node\Microsoft\Windows\CurrentVersion\Uninstall\Google Chrome" /v version }"
Current google-chrome version is UNKNOWN
Get LATEST chromedriver version for UNKNOWN google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/98.0.4758.102/chromedriver_win32.zip
Driver has been 

In [7]:
#Visit the mars nasa news site.
url = 'https://redplanetscience.com'
browser.visit(url)

#Optional delay for loading the page
#1. Searching for elements with a specific combination of tag (div) and attribute (list_text).
#2. Waiting 1 second in between searching for components because pages can have a lot of data and be image heavy
#(ethical scraping, doesn't overload server, already checked terms of service on redplanetscience.com)

browser.is_element_present_by_css('div.list_text', wait_time = 1)


True

In [8]:
#Set up the html parser
html = browser.html

news_soup = soup(html, 'html.parser')

#The . is used for selecting classes, such as list_text, so the code 'div.list_text' pinpoints the <div /> tag 
#with the class of list_text. CSS works from right to left, such as returning the last item on the list instead of the first. 
#Because of this, when using select_one, the first matching element returned will be a <li /> element 
#with a class of slide and all nested elements within it.

slide_elem = news_soup.select_one('div.list_text')


In [10]:
#Chained .find onto our previously assigned variable, slide_elem. 
#This variable holds a ton of information, so look inside of that information to find this specific data.
#The specific data is in a <div /> with a class of 'content_title'."

slide_elem.find('div', class_='content_title')

<div class="content_title">Mars Helicopter Attached to NASA's Perseverance Rover</div>

In [11]:
#Let's get rid of the html stuff, and only get the title text
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"Mars Helicopter Attached to NASA's Perseverance Rover"

In [14]:
#Now do the same thing for the summaries instead of the titles
#Some notes: .find() is used when we want only the first class and attribute we've specified.
#.find_all() is used when we want to retrieve all of the tags and attributes.
#If we were to use .find_all() instead of .find() when pulling the summary, 
#we would retrieve all of the summaries on the page instead of just the first one.

news_p = slide_elem.find('div', class_="article_teaser_body").get_text()
news_p

"The team also fueled the rover's sky crane to get ready for this summer's history-making launch."

### Featured Images

In [19]:
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [20]:
# Find a click the "full image" button on the spaceimages-mars.com page
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [24]:
# Parse the resulting html with soup
html = browser.html

img_soup = soup(html, 'html.parser')

In [27]:
# We can't pull the src link directly, because we want to pull the newest image on the website
# We will instead have to use a distinguishing attribute
# Here's the html code for reference: <img class="fancybox-image" src="image/featured/mars1.jpg" alt="">
#Note, this is a relative link.

img_url_rel = img_soup.find('img', class_="fancybox-image").get('src')

In [28]:
# If we copy and paste this link into a browser, it won't work. 
# This is because it's only a partial link, as the base URL isn't included.
# Let's get the base URL from the address bar and create an absolute link.

img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

# We're using an f-string for this print statement because it's a cleaner way to create print statements; 
# they're also evaluated at run-time. This means that it, and the variable it holds, doesn't exist until 
# the code is executed and the values are not constant. 
# This works well for our scraping app because the data we're scraping is live and will be updated frequently.

'https://spaceimages-mars.com/image/featured/mars3.jpg'

### Facts from Table

In [30]:
# Now we're going to scrape a table of facts from a different website
# Tables in HTML are basically made up of many smaller containers. 
# The main container is the <table /> tag. 
# Inside the table is <tbody />, which is the body of the table—the headers, columns, and rows.
# <tr /> is the tag for each table row. Within that tag, the table data is stored in <td /> tags. 
# This is where the columns are established.

# Instead of scraping each row, or the data in each <td />, 
# we're going to scrape the entire table with Pandas' .read_html() function.

# The Pandas function read_html() specifically searches for and returns a list of tables found in the HTML. 
# By specifying an index of 0, we're telling Pandas to pull only the first table it encounters, 
# or the first item in the list. Then, it turns the table into a DataFrame.

import pandas as pd

df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['Description','Mars', 'Earth']
df.set_index('Description', inplace=True)
df

,Mars,Earth
Description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [31]:
# Pandas also has a way to easily convert our DataFrame back into HTML-ready code using the .to_html() function.

df.to_html()


'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [32]:
# End the automated browsing session. This is an important to add to our web app also. 
# Without it, the automated browser won't know to shut down
# It will continue to listen for instructions and use the computer's resources 
# (it may put a strain on memory or a laptop's battery if left on). 

browser.quit()

In [ ]:
# We can't automate the scraping using the Jupyter Notebook. 
# To fully automate it, it will need to be converted into a .py file.
# After downloading as .py, we will need to clean up the code in a text editor. 
